In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential
from tensorflow.keras.datasets import mnist
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
# Preprocess the data
X_train = X_train.reshape((60000, 28, 28, 1))
X_test = X_test.reshape((10000, 28, 28, 1))
X_train, X_test= X_train / 255.0, X_test / 255.0

In [ ]:
# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10))


# Compile and train the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_test,  y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
print('Test accuracy:', test_acc)

Epoch 1/10
1875/1875 [==============================] - 62s 32ms/step - loss: 0.1504 - accuracy: 0.9529 - val_loss: 0.0522 - val_accuracy: 0.9835
Epoch 2/10
1875/1875 [==============================] - 50s 27ms/step - loss: 0.0454 - accuracy: 0.9855 - val_loss: 0.0337 - val_accuracy: 0.9881
Epoch 3/10
1875/1875 [==============================] - 51s 27ms/step - loss: 0.0329 - accuracy: 0.9898 - val_loss: 0.0358 - val_accuracy: 0.9894
Epoch 4/10
1875/1875 [==============================] - 51s 27ms/step - loss: 0.0257 - accuracy: 0.9919 - val_loss: 0.0393 - val_accuracy: 0.9883
Epoch 5/10
1875/1875 [==============================] - 50s 26ms/step - loss: 0.0204 - accuracy: 0.9937 - val_loss: 0.0333 - val_accuracy: 0.9907
Epoch 6/10
1875/1875 [==============================] - 51s 27ms/step - loss: 0.0159 - accuracy: 0.9949 - val_loss: 0.0404 - val_accuracy: 0.9891
Epoch 7/10
1875/1875 [==============================] - 51s 27ms/step - loss: 0.0149 - accuracy: 0.9952 - val_loss: 0.0312 -

In [ ]:
# Perform k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracy = []

for train_index, val_index in kfold.split(X_train):
   x_train_fold, x_val_fold = X_train[train_index], X_train[val_index]
   y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

In [ ]:
model.fit(x_train_fold, y_train_fold, epochs=5, batch_size=64, validation_data=(x_val_fold, y_val_fold))

Epoch 1/5
750/750 [==============================] - 40s 53ms/step - loss: 0.0035 - accuracy: 0.9987 - val_loss: 0.0021 - val_accuracy: 0.9992
Epoch 2/5
750/750 [==============================] - 38s 50ms/step - loss: 8.0774e-04 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9996
Epoch 3/5
750/750 [==============================] - 39s 52ms/step - loss: 0.0014 - accuracy: 0.9996 - val_loss: 0.0060 - val_accuracy: 0.9978
Epoch 4/5
750/750 [==============================] - 38s 51ms/step - loss: 0.0061 - accuracy: 0.9980 - val_loss: 0.0070 - val_accuracy: 0.9977
Epoch 5/5
750/750 [==============================] - 37s 49ms/step - loss: 0.0024 - accuracy: 0.9993 - val_loss: 0.0049 - val_accuracy: 0.9982


In [ ]:
_, accuracy = model.evaluate(x_val_fold, y_val_fold)
fold_accuracy.append(accuracy)

375/375 [==============================] - 3s 8ms/step - loss: 0.0049 - accuracy: 0.9982


In [ ]:
# Calculate average accuracy across folds
average_accuracy = np.mean(fold_accuracy)
print(f'Average Validation Accuracy: {average_accuracy}')

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy}')

Average Validation Accuracy: 0.9981666803359985
313/313 [==============================] - 3s 11ms/step - loss: 0.0385 - accuracy: 0.9923
Test Accuracy: 0.9922999739646912
